# Simple MNIST convnet

This example shows how to push your Keras model to the Hugging Face Hub and load the model from Hub.

**Original Author of Example:** [fchollet](https://twitter.com/fchollet)<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.

## Setup

In [19]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

🤗 Install Hugging Face Hub

In [ ]:
!pip install huggingface_hub

In [21]:
import huggingface_hub
from huggingface_hub import notebook_login, push_to_hub_keras, from_pretrained_keras

## Prepare the data

In [33]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [36]:
x_test[0].shape

(28, 28, 1)

## Build the model

In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

## Train the model

In [12]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [14]:
log_dir = "logs/fit/"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,
          callbacks = [tensorboard_callback])

Epoch 1/5
422/422 [==============================] - 47s 109ms/step - loss: 0.0274 - accuracy: 0.9909 - val_loss: 0.0292 - val_accuracy: 0.9923
Epoch 2/5
422/422 [==============================] - 44s 105ms/step - loss: 0.0273 - accuracy: 0.9907 - val_loss: 0.0280 - val_accuracy: 0.9917
Epoch 3/5
422/422 [==============================] - 43s 102ms/step - loss: 0.0263 - accuracy: 0.9913 - val_loss: 0.0262 - val_accuracy: 0.9937
Epoch 4/5
422/422 [==============================] - 42s 100ms/step - loss: 0.0242 - accuracy: 0.9916 - val_loss: 0.0260 - val_accuracy: 0.9927
Epoch 5/5
422/422 [==============================] - 43s 102ms/step - loss: 0.0242 - accuracy: 0.9917 - val_loss: 0.0311 - val_accuracy: 0.9917


We will push our model to the Hugging Face Hub with tensorboard logs.

If you already have access to keras-io organization, you can give "keras-io/{model-name}" as the model ID. If not, you can push model to your own account and then carry it to the keras-io organization later. 🥳

To push your models to the Hub, you need authentication. To authenticate, you can log using notebook_login. You can get your token from https://huggingface.co/settings/tokens 🙌🏻

In [9]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Now we can push our model to the Hugging Face Hub 🤩🙌🏻 
The below function will:

1. create a remote repository on Hugging Face Hub,
2. serialize our model,
3. create a model card including training hyperparameters, model architecture and couple of fields you can fill about model,
4. push the model to the Hub.

In [16]:
push_to_hub_keras(model, "merve/mnist", log_dir = "logs/fit/", tags = ["image-classification"])

Cloning https://huggingface.co/merve/mnist into local empty directory.


INFO:tensorflow:Assets written to: merve/mnist/assets


Adding files tracked by Git LFS: ['model.png', 'variables/variables.data-00000-of-00001', 'variables/variables.index']. This may take a bit of time if the files are large.


Upload file variables/variables.data-00000-of-00001:   2%|2         | 3.34k/141k [00:00<?, ?B/s]

Upload file saved_model.pb:   4%|3         | 3.34k/93.9k [00:00<?, ?B/s]

...

Upload file logs/train/events.out.tfevents.1654267213.183bd02d6edd.59.0.v2:   5%|4         | 3.34k/70.8k [00:0…

Upload file model.png:  15%|#5        | 3.34k/22.0k [00:00<?, ?B/s]

Upload file variables/variables.index: 100%|##########| 751/751 [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/merve/mnist
   0185bf0..5e18598  main -> main



'https://huggingface.co/merve/mnist/commit/5e18598381126b4b46d8a7507c390d8127f577da'

See pushed model with TensorBoard and model card [here](https://huggingface.co/merve/mnist).

Let's load the model!

In [24]:
model = from_pretrained_keras("merve/mnist")

config.json not found in HuggingFace Hub


## Evaluate the trained model

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.022792112082242966
Test accuracy: 0.9919999837875366
